<a href="https://colab.research.google.com/github/ricardobizerra/data-analysis/blob/main/predicting_titanic's_survivors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [157]:
import pandas as pd
import numpy as np
import torch

In [158]:
df = pd.read_csv('train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [159]:
# columns that are not relevant for the analysis are removed
df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis='columns', inplace=True)
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [160]:
df.size

7128

In [161]:
# people that were 35
df[df['Age'] == 35]

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
20,0,2,male,35.0,0,0,26.0000,S
211,1,2,female,35.0,0,0,21.0000,S
230,1,1,female,35.0,1,0,83.4750,S
258,1,1,female,35.0,0,0,512.3292,C
269,1,1,female,35.0,0,0,135.6333,S
279,1,3,female,35.0,1,1,20.2500,S
363,0,3,male,35.0,0,0,7.0500,S
383,1,1,female,35.0,1,0,52.0000,S


In [162]:
# removing empty or NaN data
df.dropna(inplace=True)
df = df[df['Fare'] != 0]
df.size

5640

In [163]:
df['AgeNorm'] = np.log10(df['Age'])
df['FareNorm'] = np.log10(df['Fare'])
df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,AgeNorm,FareNorm
0,0,3,male,22.0,1,0,7.2500,S,1.342423,0.860338
1,1,1,female,38.0,1,0,71.2833,C,1.579784,1.852988
2,1,3,female,26.0,0,0,7.9250,S,1.414973,0.898999
3,1,1,female,35.0,1,0,53.1000,S,1.544068,1.725095
4,0,3,male,35.0,0,0,8.0500,S,1.544068,0.905796
...,...,...,...,...,...,...,...,...,...,...
885,0,3,female,39.0,0,5,29.1250,Q,1.591065,1.464266
886,0,2,male,27.0,0,0,13.0000,S,1.431364,1.113943
887,1,1,female,19.0,0,0,30.0000,S,1.278754,1.477121
889,1,1,male,26.0,0,0,30.0000,C,1.414973,1.477121


In [164]:
# data normalization, in order to train a model
df['FirstClass'] = np.where(df['Pclass'] == 1, 1, 0)
df['SecondClass'] = np.where(df['Pclass'] == 2, 1, 0)

df['EmbarkedC'] = np.where(df['Embarked'] == 'C', 1, 0)
df['EmbarkedQ'] = np.where(df['Embarked'] == 'Q', 1, 0)

df['SexNorm'] = np.where(df['Sex'] == 'female', 1, 0)

df['Constant'] = 1

df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,AgeNorm,FareNorm,FirstClass,SecondClass,EmbarkedC,EmbarkedQ,SexNorm,Constant
0,0,3,male,22.0,1,0,7.2500,S,1.342423,0.860338,0,0,0,0,0,1
1,1,1,female,38.0,1,0,71.2833,C,1.579784,1.852988,1,0,1,0,1,1
2,1,3,female,26.0,0,0,7.9250,S,1.414973,0.898999,0,0,0,0,1,1
3,1,1,female,35.0,1,0,53.1000,S,1.544068,1.725095,1,0,0,0,1,1
4,0,3,male,35.0,0,0,8.0500,S,1.544068,0.905796,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
885,0,3,female,39.0,0,5,29.1250,Q,1.591065,1.464266,0,0,0,1,1,1
886,0,2,male,27.0,0,0,13.0000,S,1.431364,1.113943,0,1,0,0,0,1
887,1,1,female,19.0,0,0,30.0000,S,1.278754,1.477121,1,0,0,0,1,1
889,1,1,male,26.0,0,0,30.0000,C,1.414973,1.477121,1,0,1,0,0,1


In [165]:
coeff = torch.randn(9).double()
coeff

tensor([-0.2396,  1.9070,  1.1379,  0.3845,  1.9192,  1.9854, -1.4033, -0.9057,
         1.3863], dtype=torch.float64)

In [166]:
data = torch.tensor(df[['AgeNorm','SibSp','Parch','FareNorm','FirstClass','SecondClass','EmbarkedC','EmbarkedQ', 'Constant']].values)
data

tensor([[1.3424, 1.0000, 0.0000,  ..., 0.0000, 0.0000, 1.0000],
        [1.5798, 1.0000, 0.0000,  ..., 1.0000, 0.0000, 1.0000],
        [1.4150, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 1.0000],
        ...,
        [1.2788, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 1.0000],
        [1.4150, 0.0000, 0.0000,  ..., 1.0000, 0.0000, 1.0000],
        [1.5051, 0.0000, 0.0000,  ..., 0.0000, 1.0000, 1.0000]],
       dtype=torch.float64)

In [167]:
(data * coeff)[0].sum()

tensor(3.3024, dtype=torch.float64)

In [168]:
preds = data @ coeff.T
data[preds.isinf()]

tensor([], size=(0, 9), dtype=torch.float64)

In [169]:
labels = torch.tensor(df['Survived'].values)
labels.shape

torch.Size([705])

In [170]:
loss = ((preds-labels)**2).sum()
loss

tensor(12562.0911, dtype=torch.float64)

In [171]:
coeff = torch.randn(9).double().requires_grad_()
coeff

tensor([-0.2098,  1.0770,  0.1150,  0.0118,  0.7932, -0.8622, -0.2434, -0.6643,
         0.0668], dtype=torch.float64, requires_grad=True)

In [172]:
preds = data @ coeff.T
loss = ((preds-labels)**2).sum()
loss.backward()

In [173]:
coeff.grad

with torch.no_grad():
  coeff -= coeff.grad * 0.0001
  coeff.grad.zero_()

In [174]:
# Using Gradient Descent
for i in range(10):
  preds = data @ coeff.T
  labels = torch.tensor(df['Survived'].values)
  loss = ((preds-labels)**2).sum()
  loss.backward(retain_graph=True)

  with torch.no_grad():
    coeff -= coeff.grad * 0.0001
    coeff.grad.zero_()

In [175]:
labels

tensor([0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0,
        1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0,
        0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1,
        0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1,
        0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1,
        0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0,
        0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0,

In [176]:
preds

tensor([ 0.5493,  1.0591,  0.2355,  1.2528,  0.2378,  0.9398,  1.1100,  0.1113,
        -0.1860,  0.4810,  0.9339,  0.2337,  0.2484,  0.2308, -0.3008,  0.7882,
         0.5606, -0.2994, -0.3063, -0.4093,  0.9314,  1.1202,  0.2482,  1.7679,
         0.7344, -0.3035,  1.0582,  1.2539,  0.2340,  0.8725,  0.3530,  0.5563,
         0.0125, -0.3218, -0.4077,  0.5565,  1.4412,  0.2337,  1.0616,  0.0151,
         0.6819, -0.3119,  0.0381, -0.1248,  1.6982,  0.0362,  1.2590,  1.0540,
        -0.3095,  0.2334,  1.3684,  0.8682, -0.3088,  1.7010, -0.2932,  0.3600,
         0.2559,  0.2348, -0.4535,  0.2409,  0.2355,  0.2380,  0.9341, -0.3134,
         1.1917,  0.3704,  1.7693,  0.2350,  0.2364,  0.2335,  1.2562,  0.4329,
         0.2406,  0.7408,  0.6745, -0.3647,  0.0163,  0.2360,  0.8729,  0.2381,
         0.8699,  0.2360,  0.2336,  0.2382,  0.9388,  0.3557,  0.2344,  0.5515,
         0.0410,  0.2339, -0.3983,  0.0131,  0.6879,  1.3659,  0.3387, -0.1798,
        -0.3066,  0.8798,  0.3519,  0.23